In [1]:
#Intergers and Floats
answer=42
pi=3.14159
int(pi)
#strings
'Hello World'=="Hello World"
'hello'.capitalize()
'hello'.replace("e","a")
'hello'.isalpha()
"hell0".isdigit()
"Some,csv,values".split(",")
"Nice to meet you {0}".format("David")
name="David Yang"
f"Nice to meet you {name}"


'Nice to meet you David Yang'

In [149]:
import re

s ="dim.project"
s="\[*{0}\]*\.\[*{1}\]*".format(s.split(".")[0],s.split(".")[1])
print(s)
d = "\[*dim\]*\.\[*project\]*"
line="FROM Dim.Project".lower()
print(line)

matchObj = re.search( r'{0}[)|<|\s]|{0}$'.format(d.lower()), line,re.M)
print(matchObj!=None)
if matchObj:
    print("matched")
else:
    print("not matched")



\[*dim\]*\.\[*project\]*
from dim.project
True
matched


In [150]:
import re
import os

def containstring(content,searchedstring):
    searchedstring="\[*{0}\]*\.\[*{1}\]*".format(searchedstring.split(".")[0],searchedstring.split(".")[1])
    matchObj = re.search( r'{0}[)|<|\s]|{0}$'.format(searchedstring.lower()), content.lower(), re.M)
    return matchObj!=None

def loadfromfile(filepath):
    file = open(filepath, "r") 
    return file.read()

def convertfilenametotable(filename):
    if filename.find(".DTSX")>0:
        return filename.upper().replace(".DTSX","").replace("STG_","STG.").replace("FACT_","FACT.").replace("DIM_","DIM.")
    else:
        return ""

def dependencylevel(origfile,linkedfile,result):
    if len(result[linkedfile]["linkedTables"])>0:
        result[origfile]["level"]= result[origfile]["level"]+1
        for tb in result[linkedfile]["linkedTables"]:
            for key in result.keys():
                if result[key]["tablename"].upper()==tb.upper():
                    dependencylevel(origfile,key,result)
                    break

def readSSISfilesInfo(ETLpath,groupname):
    groupfiles = [f.upper() for f in os.listdir(ETLpath) if 
             os.path.isfile(os.path.join(ETLpath, f)) 
             and ".DTSX" in f.upper() 
             and groupname.upper() in f.upper()]
    
    result={}
    for filename in groupfiles:
        tablename=convertfilenametotable(filename)
        if  tablename !="":
            result[filename]={}
            result[filename]["tablename"]=tablename
            result[filename]["linkedTables"]=[]
            result[filename]["level"]=0
    return result

### find out all linked tables for each SSIS package
def getlinkedtables(SSISFiles):
   for filename in SSISFiles.keys():
    content=loadfromfile(os.path.join(ETLpath,filename))
    owntablename= SSISFiles[filename]["tablename"]
    SSISFiles[filename]["linkedTables"].clear()
    ### print(".",end=" ")
    for file in SSISFiles.keys():
        tablename=SSISFiles[file]["tablename"]
        if  tablename.upper()!=owntablename.upper() and containstring(content,tablename) :
            SSISFiles[filename]["linkedTables"].append(tablename) 

### generate layers for each SSIS package
def generatelayers(ETLpath,groupname):
    SSISFiles=readSSISfilesInfo(ETLpath,groupname)
    getlinkedtables(SSISFiles)
    for filename in SSISFiles.keys():
        if SSISFiles[filename]["level"]==0:
            dependencylevel(filename,filename,SSISFiles)
    return SSISFiles


In [153]:
import csv
ETLpath="C:\projects"
groupname="FACT"
result = generatelayers(ETLpath,groupname)

In [155]:
with open(r'C:\temp\Fact.csv','w') as f:
    for k,v in result.items():
        f.write("{0}\t{1}\t{2}\t{3}\n".format(k,v["tablename"],
                  "" if len(v["linkedTables"])==0 else str(v["linkedTables"]),int(v["level"])))
   

In [154]:
### generate layers for each SSIS package
print("Dependency")
for filename in result.keys():
    if len(result[filename]["linkedTables"])>0:
        print("{0} depends on {1} and level={2}".format
              (filename,result[filename]["linkedTables"],result[filename]["level"]) )
        
    


Dependency
FACT_ADJUSTEDENERGYBUDGETS.DTSX depends on ['FACT.ENERGYREGRESSIONS', 'FACT.WEATHERSTATIONDATA'] and level=1
FACT_ADJUSTEDENERGYCONSUMPTIONS.DTSX depends on ['FACT.DAILYENERGYINVOICES', 'FACT.ENERGYREGRESSIONS', 'FACT.WEATHERSTATIONDATA'] and level=2
FACT_ADJUSTEDYEARENERGYCONSUMPTIONS.DTSX depends on ['FACT.ENERGYREGRESSIONS', 'FACT.WEATHERSTATIONDATA'] and level=1
FACT_ASSETPROJECTS.DTSX depends on ['FACT.ASSETCOSTS'] and level=1
FACT_BJCCCOSTS.DTSX depends on ['FACT.WORKORDERS'] and level=3
FACT_BUDGETSFORECASTS.DTSX depends on ['FACT.OMBUDGETS'] and level=1
FACT_CAPITALPROJECTS_VENDORCSAT.DTSX depends on ['FACT.CAPITALPROJECTSCSAT'] and level=1
FACT_CAPITALPROJECTTASKS.DTSX depends on ['FACT.CAPITALPROJECTS'] and level=1
FACT_DAILYENERGYCOSTS.DTSX depends on ['FACT.BJCCCOSTS'] and level=4
FACT_DAILYENERGYFORECAST.DTSX depends on ['FACT.ENERGYFORECASTRATES', 'FACT.ENERGYINVOICELINES'] and level=3
FACT_DAILYENERGYINVOICES.DTSX depends on ['FACT.ENERGYINVOICELINES'] and lev

In [ ]:
"""
###Unit Test
testdata ={"A.dtsx":{"tablename":"A",linkedTables":["B","C"],"level":0},
           "B.dtsx":{"tablename":"B","tablename":"A","linkedTables":["D"],"level":0},
           "C.dtsx":{"tablename":"C","linkedTables":["F"],"level":0},
           "D.dtsx":{"tablename":"D","linkedTables":["E"],"level":0},
           "E.dtsx":{"tablename":"E","linkedTables":["F"],"level":0},
           "F.dtsx":{"tablename":"F","linkedTables":[],"level":0},
          }
for tablename in testdata.keys():
    if testdata[tablename]["level"]==0:
        dependencylevel(tablename,tablename,testdata)
        
print(testdata)
 """    